In [22]:
import os
import json
from PIL import Image
import xml.etree.ElementTree as ET

def add_boxes(json_file, img_dir, xml_dir):
    with open(json_file, "r") as f:
        data = json.load(f)
    cats = data["categories"]
    name_to_id = {c["name"]: c["id"] for c in cats}
    images_dir = os.listdir(img_dir)
    annotations = []
    images = []
    img_id=0

    for fname in images_dir:
        img_id+=1
        if not fname.endswith(".jpg"):
            continue
        base = os.path.splitext(fname)[0]
        image_path = os.path.join(img_dir, fname)
        try:
            with Image.open(image_path) as img:
                width, height = img.size
        except Exception as e:
            print(f"Failed to open image {image_path}: {e}")
            continue
        images.append({
            "id": img_id,
            "file_name": fname,
            "width": width,
            "height": height
        })
        xml_path = os.path.join(xml_dir, base + ".xml")
        if not os.path.exists(xml_path):
            annotations.append({"image_id": img_id, "annotations": []})
            continue
        tree = ET.parse(xml_path)
        root = tree.getroot()
        img_anns = []
        for obj in root.findall("object"):
            if obj.find("name") is None:
                continue
            name = obj.find("name").text
            cat_id = name_to_id.get(name)
            if cat_id is None:
                continue
            box = obj.find("bndbox")
            x = float(box.find("xmin").text)
            y = float(box.find("ymin").text)
            xmax = float(box.find("xmax").text)
            ymax = float(box.find("ymax").text)
            w = xmax - x
            h = ymax - y
            img_anns.append({"category_id": cat_id, "bbox": [x, y, w, h]})
        annotations.append({"image_id": img_id, "annotations": img_anns})
    data["annotations"] = annotations
    data["images"] = images
    with open(json_file, "w") as f:
        json.dump(data, f, indent=2)
    print(f"Updated {json_file} with {len(annotations)} image annotations and {len(images)} image entries.")


In [24]:
add_boxes('train.json','../subset/train/images','../subset/train/annotations' )

Updated train.json with 74960 image annotations and 74960 image entries.


In [25]:
add_boxes('test.json','../subset/test/images','../subset/test/annotations' )

Updated test.json with 13310 image annotations and 13310 image entries.
